# Predict dual models

## Import libraries

In [1]:
import os
import glob
import re
import shutil
import sys
import warnings

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cv2
from PIL import Image
from tqdm import tqdm

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import load_model
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

from models import unetpp_level_1, unet_level_2

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
print("Tensorflow version: {}".format(tf.__version__))

Tensorflow version: 2.1.0


In [3]:
from tensorflow.python.client import device_lib
print("GPU sample processing: ")
print(device_lib.list_local_devices())

GPU sample processing: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12977154852435322817
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9992663860
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10846868465568318858
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


## Setup global variables

In [4]:
IMG_HEIGHT = 912
IMG_WIDTH = 912
CROP_HEIGHT = 228
CROP_WIDTH = 228

BURNED_PIXEL_VALUE = 1

In [5]:
PATH_WEIGHT_NETWORK_1 = "C:/Users/windows/Desktop/Research/3. Code/0.GITHUB-CODE/forest-fire-damage-mapping/[Project]_Model_weights/network_1_weights/checkpoint"
PATH_WEIGHT_NETWORK_2 = "C:/Users/windows/Desktop/Research/3. Code/0.GITHUB-CODE/forest-fire-damage-mapping/[Project]_Model_weights/network_2_weights/checkpoint"

In [6]:
PATH_TESTING_1 = "C:/Users/windows/Desktop/Research/3. Code/0.GITHUB-CODE/forest-fire-damage-mapping/sample_data/sample_location_2_data/Img/"
PATH_MASK_TESTING_1 = "C:/Users/windows/Desktop/Research/3. Code/0.GITHUB-CODE/forest-fire-damage-mapping/sample_data/sample_location_2_data/Label/"

PATH_TESTING_2 = "C:/Users/windows/Desktop/Research/3. Code/6. UnetFire/ImgLabel/official_data_10.6/location_2_crop_for_2nd_network/Img/"

In [7]:
PATH_SAVE_PREDICTION = "C:/Users/windows/Desktop/Research/3. Code/0.GITHUB-CODE/forest-fire-damage-mapping/sample_data/sample_location_2_data/"

### Load models 

In [8]:
def load_model(model_level, path_weight, model_summary = False):
    if model_level == 1:
        model = unetpp_level_1.create_model()
        model.load_weights(path_weight)
    elif model_level == 2:
        model = unet_level_2.create_model()
        model.load_weights(path_weight)
    if model_summary:
        model.summary()
    return model

In [9]:
model_level_1 = load_model(model_level = 1, path_weight = PATH_WEIGHT_NETWORK_1, model_summary = False)

In [10]:
model_level_2 = load_model(model_level = 2, path_weight = PATH_WEIGHT_NETWORK_2, model_summary = False)

In [11]:
alpha = 0.25
gamma = 2
def focal_loss_with_logits(logits, targets, alpha, gamma, y_pred):
    weight_a = alpha * (1 - y_pred) ** gamma * targets
    weight_b = (1 - alpha) * y_pred ** gamma * (1 - targets)

    return (tf.math.log1p(tf.exp(-tf.abs(logits))) + tf.nn.relu(
        -logits)) * (weight_a + weight_b) + logits * weight_b

def focal_loss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(),
                              1 - tf.keras.backend.epsilon())
    logits = tf.math.log(y_pred / (1 - y_pred))

    loss = focal_loss_with_logits(logits=logits, targets=y_true,
                                  alpha=alpha, gamma=gamma, y_pred=y_pred)
    return tf.reduce_mean(loss)

### Evaluation metrics

In [12]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (
                K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(
        K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [13]:
def dice_loss_final(y_true, y_pred):
    smooth = 1.

    y_true_f = y_true.reshape(-1)
    y_pred_f = y_pred.reshape(-1)
    intersection = np.sum((y_true_f * y_pred_f))
    
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

In [14]:
model_level_1.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4), loss = focal_loss, metrics =[dice_coef, sensitivity, specificity])

In [15]:
def network_1_prediction(path_img_level_1, path_mask_level_1):
    img_level_1 = load_img(path_img_level_1, grayscale=False, target_size=[IMG_HEIGHT, IMG_WIDTH])
    mask_level_1 = load_img(path_mask_level_1, grayscale=True, target_size=[IMG_HEIGHT, IMG_WIDTH])
    # Preprocessing image
    img_level_1 = img_to_array(img_level_1)
    img_level_1 = img_level_1.astype('float32')
    img_level_1 /= 255
    img_level_1 = img_level_1.reshape(1,IMG_HEIGHT,IMG_WIDTH,3)
    # Preprocessing mask
    mask_level_1 = img_to_array(mask_level_1)
    mask_level_1 = mask_level_1.astype('float32')
    mask_level_1 /= np.max(mask_level_1)
    
    where_are_NaNs = np.isnan(mask_level_1)
    mask_level_1[where_are_NaNs] = 0
    mask_level_1 = mask_level_1.reshape(1,IMG_HEIGHT,IMG_WIDTH,1)
    # Network 1 prediction
    result_level_1 = model_level_1.predict(img_level_1)
    # Reshape predicted result
    result_level_1[0,:,:,0][result_level_1[0,:,:,0] > 0.5] = 1
    result_level_1[0,:,:,0][result_level_1[0,:,:,0] <= 0.5] = 0
    result_level_1 = result_level_1.reshape(IMG_HEIGHT,IMG_WIDTH)
    # Model evaluate
    _, dice_coef, sensitivity, specificity = model_level_1.evaluate(img_level_1, mask_level_1, verbose = 0)
    return dice_coef, sensitivity, specificity, result_level_1

In [16]:
def extract_crop_num(result):
    crop_num = 1
    save_crop_burned_num = []
    for row in np.arange(0, IMG_HEIGHT, CROP_HEIGHT):
        for col in np.arange(0, IMG_WIDTH, CROP_WIDTH):
            crop_window = result[col:col+CROP_HEIGHT, row:row+CROP_WIDTH]
            if crop_window.any() == 1:
                save_crop_burned_num.append(crop_num)
            crop_num += 1
    return save_crop_burned_num

In [17]:
def network_2_fine_tune(num_img, save_crop_burned_num):
    final_prediction = np.zeros((IMG_HEIGHT, IMG_WIDTH))
    crop_num = 1
    scale_height = 128
    scale_width = 128
    for row in np.arange(0, IMG_HEIGHT, CROP_HEIGHT):
        for col in np.arange(0, IMG_WIDTH, CROP_WIDTH):
            if crop_num in save_crop_burned_num:
#                 print("Fine tune damage map")
                path_img_level_2 = PATH_TESTING_2 + 'img{}crop{}.png'.format(num_img, crop_num)
                img_level_2 = load_img(path_img_level_2, grayscale=False, target_size=[scale_height, scale_width])
                img_level_2 = img_to_array(img_level_2)
                img_level_2 = img_level_2.astype('float32')
                img_level_2 /= 255
                img_level_2 = img_level_2.reshape(1,scale_height,scale_width,3)
                # Network 2 predict path-level images
                result_level_2 = model_level_2.predict(img_level_2)
                result_level_2[0,:,:,0][result_level_2[0,:,:,0] > 0.5] = 1
                result_level_2[0,:,:,0][result_level_2[0,:,:,0] <= 0.5] = 0
                result_level_2 = result_level_2.reshape(scale_height,scale_width)
                # Resize 
                result_level_2_resize = cv2.resize(result_level_2, (CROP_HEIGHT, CROP_WIDTH), interpolation=cv2.INTER_LINEAR)
                final_prediction[col:col+CROP_HEIGHT, row:row+CROP_WIDTH] = result_level_2_resize
            elif crop_num not in save_crop_burned_num:
                final_prediction[col:col+CROP_HEIGHT, row:row+CROP_WIDTH] = np.zeros(final_prediction[col:col+CROP_HEIGHT, row:row+CROP_WIDTH].shape)
            crop_num += 1
    return final_prediction

In [18]:
def dual_models_prediction(num_testing= 720, visualize_compare_results = True, save_prediction = False):
    dice_coef_save_level_1 = []
    sensitivity_save_level_1 = []
    specificity_save_level_1 = []
    
    dice_coef_save_final= []
    sensitivity_save_final = []
    specificity_save_final = []
    for num_img in tqdm (range(697, num_testing+1)):
#     for num_img in tqdm (range(121, num_testing+1)):
#     for num_img in tqdm (np.array([num_testing])):
        # Load testing image for level 1 network
        path_img_level_1 = PATH_TESTING_1 + "img{}.png".format(num_img) 
        path_mask_level_1 = PATH_MASK_TESTING_1 + "label{}.png".format(num_img) 
        
        img_level_1 = load_img(path_img_level_1, grayscale=False, target_size=[IMG_HEIGHT, IMG_WIDTH])
        mask_level_1 = load_img(path_mask_level_1, grayscale=True, target_size=[IMG_HEIGHT, IMG_WIDTH])
        mask_level_1 = img_to_array(mask_level_1)
        mask_level_1 = mask_level_1.astype('float32')
        
#         print(np.unique(mask_level_1))

        mask_level_1 /= np.max(mask_level_1)
        where_are_NaNs = np.isnan(mask_level_1)
        mask_level_1[where_are_NaNs] = 0
        
        mask_level_1 = mask_level_1.reshape(IMG_HEIGHT,IMG_WIDTH)
        
        if visualize_compare_results:
            plt.figure(figsize=(10,5))
            plt.subplot(1,2,1)
            plt.imshow(img_level_1)
            plt.title('Input testing images')
        # Network 1 predict 
        dice_coef, sensitivity, specificity, result_level_1 = network_1_prediction(path_img_level_1,path_mask_level_1)
#         print(result_level_1.shape)
        # Save model evaluation metrics
        dice_coef_save_level_1.append(dice_coef)
        sensitivity_save_level_1.append(sensitivity)
        specificity_save_level_1.append(specificity)
        # Extract crop window with burned pixels
        save_crop_burned_num = extract_crop_num(result_level_1)
        # Network 2 fine tune
        final_prediction = network_2_fine_tune(num_img, save_crop_burned_num)
        final_prediction[final_prediction > 0.5] = 1
        final_prediction[final_prediction <= 0.5] = 0
        # Visualize
        if visualize_compare_results:
            
            final_result = np.ma.masked_where(final_prediction == BURNED_PIXEL_VALUE, final_prediction)
            cmap = matplotlib.cm.Greys  
            cmap.set_bad(color='black')
            plt.subplot(1,2,2)
            plt.imshow(final_result, cmap = cmap)
            plt.title('Dual model prediction')
            plt.show()
        # Save prediction
        if save_prediction:
            path_save = PATH_SAVE_PREDICTION + "predict{}.png".format(num_img)
            cv2.imwrite(path_save,final_prediction*255)
            
        tn_, fp_, fn_, tp_ = confusion_matrix(mask_level_1.reshape(-1), final_prediction.reshape(-1), labels=[0,1]).ravel()
        specificity = tn_ / (tn_+fp_)
        sensitivity = tp_ / (tp_+fn_)
        dice_coef_final =  dice_loss_final(mask_level_1, final_prediction)
        
        dice_coef_save_final.append(dice_coef_final)
        sensitivity_save_final.append(sensitivity)
        specificity_save_final.append(specificity)
        
    return dice_coef_save_final, sensitivity_save_final, specificity_save_final

In [19]:
dice_coef_save_final, sensitivity_save_final, specificity_save_final = dual_models_prediction(num_testing=720, visualize_compare_results = False, save_prediction = False)

100%|██████████| 24/24 [01:42<00:00,  4.29s/it]


In [20]:
sensitivity_save_final = np.array(sensitivity_save_final)
where_are_NaNs = np.isnan(sensitivity_save_final)
sensitivity_save_final[where_are_NaNs] = 0

specificity_save_final = np.array(specificity_save_final)
where_are_NaNs = np.isnan(specificity_save_final)
specificity_save_final[where_are_NaNs] = 0

print("Performance of final ")
print("Dice coef = {}".format(np.mean(np.array(dice_coef_save_final))))
print("Sensitivity = {}".format(np.mean(np.array(sensitivity_save_final))))
print("Specificity = {}".format(np.mean(np.array(specificity_save_final))))

Performance of final 
Dice coef = 0.789062636574695
Sensitivity = 0.1349067774682086
Specificity = 0.9855287802647078
